In [2]:
%load_ext autoreload
%autoreload 2

import os

import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import LineString
from shapely import wkt
import numpy as np
import swifter
from generator.preprocess import *
from generator.preprocess import remove_outlier_trajectories
from generator.road_network import RoadNetwork
import glob
from tqdm import tqdm

In [3]:
df = pd.read_csv("../datasets/trajectories/hanover/hannover_inrix.csv")
network = RoadNetwork()
network.load("../osm_data/hanover")

In [ ]:
l, c = 0, 0
for _, g in tqdm(df.groupby("id")):
    if g.shape[0] < 5:
        continue
    l += g.shape[0]
    c += 1

print(l/c)


100%|██████████| 77188/77188 [00:02<00:00, 34545.13it/s]

241.32424353095539


In [ ]:
rows = []
for _, g in tqdm(df.groupby("id")):
    if g.shape[0] < 5:
        continue
    traj = g[["longitude", "latitude", "time"]].copy()
    traj.loc[:, "time"] = pd.to_datetime(traj["time"], infer_datetime_format=True).dt.floor('s')
    traj.loc[:, "time"] = traj["time"].values.astype(np.int64) // 10 ** 9
    
    data = traj.to_numpy()
    time = data[1:, 2] - data[0, 2]
    time = np.insert(time, 0, 0)
    seq = LineString(data[:, :2])
    assert len(list(seq.coords)) == time.shape[0]
    rows.append((seq, time.astype(int).tolist(), traj.iloc[0, 2], traj.iloc[-1, 2]))

processed_df = pd.DataFrame(rows, columns=["POLYLINE", "timestamp", "start_stamp", "end_stamp"])
processed_df

100%|██████████| 77188/77188 [03:10<00:00, 404.68it/s]


,POLYLINE,timestamp,start_stamp,end_stamp
0,"LINESTRING (9.8288 52.3326, 9.819705 52.331356...","[0, 80, 424, 535, 613, 1094, 1171, 1453, 1528,...",1565974403,1565982018
1,"LINESTRING (9.6472 52.2823, 9.643848 52.283978...","[0, 30, 60, 90, 120, 172, 202, 232, 262, 292, ...",1567075203,1567075868
2,"LINESTRING (9.7376 52.5465, 9.72797 52.541855,...","[0, 138, 300, 485, 668, 801, 882]",1566184436,1566185318
3,"LINESTRING (9.7119 52.343, 9.716048 52.344969,...","[0, 90, 180, 229, 319, 409, 492, 582, 672, 762]",1568621637,1568622399
4,"LINESTRING (9.7585 52.3363, 9.762929 52.34235,...","[0, 79, 141, 224, 377, 453, 580, 685, 763, 881...",1566671810,1566673656
...,...,...,...,...
73924,"LINESTRING (10.0369 53.3593, 10.037012 53.3582...","[0, 5, 8, 14, 15, 19, 25, 28, 30, 34, 39, 44, ...",1569563606,1569569649
73925,"LINESTRING (9.7278 52.3654, 9.727219 52.365731...","[0, 5, 8, 16, 19, 20, 25, 29, 35, 40, 46, 49, ...",1575029942,1575031187
73926,"LINESTRING (9.8726 52.9458, 9.718267 52.63466,...","[0, 599, 703, 718, 733, 748, 763, 778, 1295, 1...",1573224957,1573226965
73927,"LINESTRING (9.738963 52.382663, 9.738996 52.38...","[0, 5, 10, 15, 20, 25, 30, 35, 45, 50, 55, 60,...",1574319815,1574320945


In [8]:
city_bounds = network.bounds
clipped = clip_trajectories(processed_df.copy(), city_bounds, polyline_convert=True)
# df_clipped = filter_min_points(df_clipped, 5)

In [9]:
""" 
Correct timestamps
"""

def strictly_increasing(L):
    return all(x+20>=y for x, y in zip(L, L[1:]))


def correct_timestamps(traj, orig_trajs, orig_ts):
    corrected_ts = []
    corrected_traj = []
    idxs = []
    found = False
    for i, g1 in enumerate(traj):
        ridx = 0
        for j, g2 in enumerate(orig_trajs[ridx:]):
            if g1 == g2:
                found = True
                corrected_ts.append(orig_ts[j])
                corrected_traj.append(g1)
                idxs.append(j)
                ridx = j+1
                break
            # if found:
            #     break

    assert len(corrected_traj) == len(corrected_ts)
    # assert strictly_increasing(idxs), (idxs)
    
    return corrected_traj, (np.array(corrected_ts) - corrected_ts[0]).astype(int).tolist()


rows = []
i = 0
orig_polies, orig_ts = processed_df.POLYLINE, processed_df.timestamp
for i, r in tqdm(clipped.iterrows()):
    op = list(orig_polies.loc[r.name].coords)
    ot = orig_ts.loc[r.name]
    if type(r.POLYLINE) == LineString:
        traj = list(r.POLYLINE.coords)
        if len(traj) < 5:
            continue
        ctraj, cts = correct_timestamps(traj, op, ot)
        rows.append([LineString(ctraj), cts, r.start_stamp, r.end_stamp])
    else:
        for line in r.POLYLINE:
            traj = list(line.coords)
            if len(traj) < 5:
                continue
            ctraj, cts = correct_timestamps(traj, op, ot)
            rows.append([LineString(ctraj), cts, r.start_stamp, r.end_stamp])

df = pd.DataFrame(rows, columns=["POLYLINE", "timestamp", "start_stamp", "end_stamp"])

71300it [01:51, 640.69it/s] 


In [13]:
df["id"] = np.arange(1, df.shape[0]+1)
df["timestamp"] = df["timestamp"].astype(str)
df["timestamp"] = df["timestamp"].str.replace("[", "")
df["timestamp"] = df["timestamp"].str.replace("]", "")
#df_clipped["timestamp"] = df_clipped["timestamp"].str.replace("  ", ", ")
df.to_csv("../datasets/trajectories/hanover/mapped_id_poly_clipped_corrected.csv", sep=";", index=False)

In [14]:
df = pd.read_csv("../datasets/trajectories/hanover/mapped_id_poly_clipped.csv", sep=";")
df

,timestamp,start_stamp,end_stamp,POLYLINE,id
0,"0, 80, 424, 535, 613, 1094, 1171, 1453, 1528, ...",1565974403,1565982018,"LINESTRING (9.8288 52.3326, 9.819705 52.331356...",1
1,"0, 30, 60, 90, 120, 172, 202, 232, 262, 292, 5...",1567075203,1567075868,"LINESTRING (9.660448261338471 52.3136182, 9.66...",2
2,"0, 90, 180, 229, 319, 409, 492, 582, 672, 762",1568621637,1568622399,"LINESTRING (9.7119 52.343, 9.716048 52.344969,...",3
3,"0, 79, 141, 224, 377, 453, 580, 685, 763, 881,...",1566671810,1566673656,"LINESTRING (9.7585 52.3363, 9.762929 52.34235,...",4
4,"0, 1, 9, 15, 26, 30, 34, 41, 46, 51, 60, 66, 7...",1574157039,1574159903,"LINESTRING (9.6138059 52.428522640720765, 9.62...",5
...,...,...,...,...,...
143854,"0, 5, 8, 14, 15, 19, 25, 28, 30, 34, 39, 44, 4...",1569563606,1569569649,"LINESTRING (9.718658264790285 52.4474529, 9.71...",143855
143855,"0, 5, 8, 16, 19, 20, 25, 29, 35, 40, 46, 49, 5...",1575029942,1575031187,"LINESTRING (9.7278 52.3654, 9.727219 52.365731...",143856
143856,"0, 599, 703, 718, 733, 748, 763, 778, 1295, 16...",1573224957,1573226965,"LINESTRING (9.782460425078042 52.4474529, 9.78...",143857
143857,"0, 5, 10, 15, 20, 25, 30, 35, 45, 50, 55, 60, ...",1574319815,1574320945,"LINESTRING (9.738963 52.382663, 9.738996 52.38...",143858


In [15]:
df = pd.read_csv("../datasets/trajectories/hanover/mapped_id_poly_clipped_corrected.csv", sep=";")
df

,POLYLINE,timestamp,start_stamp,end_stamp,id
0,"LINESTRING (9.7597 52.3939, 9.76275 52.393017,...","0, 120, 241, 362, 483, 603, 724",1573818302,1573824822,1
1,"LINESTRING (9.7565 52.3949, 9.758446 52.395981...","0, 30, 94, 124, 154, 184, 214, 277, 307, 337, ...",1574525822,1574533547,2
2,"LINESTRING (9.7255 52.3472, 9.719058 52.342037...","0, 81, 173, 295, 445, 598, 700, 776, 879",1567141319,1567149096,3
3,"LINESTRING (9.88635 52.398609, 9.872884 52.401...","0, 30, 60, 90",1568735955,1568742534,4
4,"LINESTRING (9.883612 52.398881, 9.868577 52.40...","0, 31, 45, 136",1572780239,1572788527,5
...,...,...,...,...,...
143854,"LINESTRING (9.710756 52.447116, 9.710982 52.44...","0, 15, 29, 44, 59, 75, 89, 104, 120, 134, 149,...",1566376125,1566386730,143855
143855,"LINESTRING (9.7686 52.376, 9.768375 52.376103,...","0, 2, 3, 54, 74, 89, 103, 119, 122, 128, 134, ...",1567093909,1567105126,143856
143856,"LINESTRING (9.850325 52.413299, 9.851001 52.41...","0, 4, 9, 14, 20",1567093909,1567105126,143857
143857,"LINESTRING (9.847199 52.412983, 9.846692 52.41...","0, 4, 9, 15, 19, 24, 30, 34, 38, 44, 49, 54, 58",1567093909,1567105126,143858


In [6]:
df["POLYLINE"] = df["POLYLINE"].swifter.apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs="epsg:4326", geometry="POLYLINE")

Pandas Apply:   0%|          | 0/153576 [00:00<?, ?it/s]

In [7]:
network.fmm_trajectorie_mapping(
    
    network_file="../osm_data/hanover/edges.shp",
    input_file="../datasets/trajectories/Hanover/mapped_id_poly_clipped.csv",
    output_file="../datasets/trajectories/Hanover/road-segment-mapping.txt"
)

In [30]:
# preprocess the mapping especially the speed and distance values need to be verified
df = pd.read_csv("../datasets/trajectories/hanover/road-segment-mapping.csv", sep=";")
df_prep = remove_outlier_trajectories(df.copy(), min_edges_traversed=3, max_speed=0.000251)
df_prep.to_csv("../datasets/trajectories/hanover/road_segment_map_final.csv", sep=";")

Pandas Apply:   0%|          | 0/82406 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/82406 [00:00<?, ?it/s]

/dstore/home/pheinemeyer/Road-Network-Embedding-Generator/generator/preprocess.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[(df["speed_mean"] > max_speed)]["speed"] = df[(df["speed_mean"] > max_speed)][


Pandas Apply:   0%|          | 0/56921 [00:00<?, ?it/s]

/dstore/home/pheinemeyer/Road-Network-Embedding-Generator/generator/preprocess.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["speed_mean"] = df["speed"].swifter.apply(np.mean)


In [4]:
"""
Test of Travel Time Dataset generation
"""
from generator.trajectory import Trajectory

traj = Trajectory("../datasets/trajectories/hanover/road_segment_map_final_corrected_han.csv", nrows=1000000)

In [5]:
traj.df

,Unnamed: 0,id,opath,spdist,pgeom,cpath,mgeom,duration,speed,speed_mean
0,12,33347,"[15659, 15659, 18484, 9202]","9.85719e-05,0.00105132,0.00300252","LINESTRING(9.82732369848 52.3611495794,9.82742...","[15659, 662, 18482, 18484, 13704, 12174, 9202]","LINESTRING(9.82732369848 52.3611495794,9.82750...","30,60,30","[3.28572974605e-06, 1.75220801434e-05, 0.00010...",0.000040
1,16,66668,"[6752, 6752, 6752, 6752, 6752, 6752, 17764, 17...","0.000944413,0.000733552,0.000445603,0.00087625...","LINESTRING(9.81773680786 52.3148201013,9.81728...","[6752, 17760, 17764, 472, 17705, 17703, 17744,...","LINESTRING(9.81773680786 52.3148201013,9.81748...","6,5,3,6,6,5,3,6,6,3,5,7,5,5,5,3,7,4,6,5,5,5,5,...","[0.000157402088135, 0.000146710448545, 0.00014...",0.000240
2,17,66669,"[186, 186, 17981, 17981, 17981, 244]","0.000490697,0.0010319,0.00101119,0.000561632,0...","LINESTRING(9.79367166034 52.3515983761,9.79383...","[186, 17981, 244]","LINESTRING(9.79367166034 52.3515983761,9.79365...","6,4,6,5,3","[8.17827843251e-05, 0.00025797605376, 0.000168...",0.000166
3,23,33356,"[17118, 17118, 982, 1467, 13097, 13100, 16856,...","0.001787,0.0019207,0.00223016,0.00129155,0.001...","LINESTRING(9.79313565554 52.4175126922,9.79246...","[17118, 982, 1467, 13097, 13100, 16856, 16860,...","LINESTRING(9.79313565554 52.4175126922,9.79302...","15,15,15,30,30,15,15,15,30,15,15","[0.000119133610677, 0.000128046865169, 0.00014...",0.000106
4,30,33363,"[46, 9, 3, 3, 12232, 7441]","0.0148044,0.0219578,0.0251783,0.0208982,0.026086","LINESTRING(9.74121859741 52.4273022057,9.75601...","[46, 9, 9453, 3, 14001, 12232, 14541, 7634, 74...","LINESTRING(9.74121859741 52.4273022057,9.74127...","119,120,120,120,120","[0.000124406779548, 0.000182981907171, 0.00020...",0.000182
...,...,...,...,...,...,...,...,...,...,...
50406,143850,114612,"[10558, 10558, 10558, 10558, 10558, 10560, 105...","0.000705096,0.00038524,0.000807804,0.000367846...","LINESTRING(9.68686379794 52.3896295783,9.68683...","[10558, 10560, 11772, 1608, 15458, 1512, 14875...","LINESTRING(9.68686379794 52.3896295783,9.68684...","5,3,7,8,1,5,5,4,1,5,5,4,2,5,5,9,9,1,5,6,3,6,5,...","[0.000141019112373, 0.00012841322132, 0.000115...",0.000127
50407,143852,114614,"[463, 753, 753, 10560, 15458, 1513, 18086, 16227]","0.00235974,0.0013332,0.000693332,0.0039168,0.0...","LINESTRING(9.688578798 52.3884757376,9.6872579...","[463, 753, 10560, 11772, 1608, 15458, 1513, 15...","LINESTRING(9.688578798 52.3884757376,9.6885751...","30,30,30,30,30,49,30","[7.86578648547e-05, 4.4440050956e-05, 2.311107...",0.000099
50408,143853,114615,"[1857, 1857, 3095, 1857]","2.25033e-05,0.00556824,0.00880897","LINESTRING(9.71306781495 52.3911703044,9.71304...","[1857, 3095, 1857]","LINESTRING(9.71306781495 52.3911703044,9.71281...","100,30,30","[2.25033209673e-07, 0.000185607942539, 0.00029...",0.000160
50409,143854,114616,"[16226, 17927, 1518, 1518, 1515, 11773, 3847, ...","0.00190745,0.00422997,0.000894784,0.00325849,0...","LINESTRING(9.70120265823 52.3914199382,9.69931...","[16226, 18085, 3529, 17927, 1518, 1515, 15457,...","LINESTRING(9.70120265823 52.3914199382,9.70111...","30,30,30,30,61,60,30,30,62,30,30,30,30,66,30,30","[6.35816546071e-05, 0.000140999122008, 2.98261...",0.000153


In [6]:
"""
Generate traj features 
"""
features = traj.generate_speed_features(network)
features.to_csv("../datasets/trajectories/hanover/speed_features_unnormalized_corrected.csv")

50411it [00:30, 1653.41it/s]


In [15]:
features.describe()

,id,util,avg_speed
count,18576.000000,18576.000000,13751.000000
mean,9287.500000,42.012220,38.970848
std,5362.573636,106.154144,20.793826
min,0.000000,0.000000,0.000000
25%,4643.750000,1.000000,25.955203
50%,9287.500000,5.000000,37.289088
75%,13931.250000,31.000000,48.714460
max,18575.000000,1712.000000,194.398395


In [16]:
print(traj.df.shape)

(42357, 10)
